In [36]:
#Link checking mit csv-Datei:

#Importieren aller notwendigen Libraries:
import csv
import pandas as pd
import requests
from email.message import EmailMessage
import smtplib
from datetime import datetime

In [41]:
#Zusammenfügen von Report erstellen und E-Mail senden:
#Als erstes erstelle ich eine Funktion um eine E-Mail mit csv-Anhang an die Ziel-E-Mail-Addresse zu senden:

def send_email_alert(sender, recipient, report):
    from email.message import EmailMessage
    import smtplib
    
#header

    msg = EmailMessage()
    msg["Subject"] = "Broken Links in Reading List"
    #sender ='Theodor.sonnenschein@web.de' 
    msg['From'] = sender
    #recipient = 'Theodor.sonnenschein@web.de'
    msg['To'] = recipient

#Nachricht und Anhang
    date = datetime.now().strftime('%Y-%m-%d')
    
    msg.set_content("Dear Team, \nsome of the links are not working. Please have a look!")

    with open("broken_links.csv", "rb") as fp:
        msg.add_attachment(
            fp.read(), maintype= date + "_broken_links", subtype="csv")
    
#Verbindung

    with smtplib.SMTP('smtp.web.de', 587) as smtplib:
        smtplib.ehlo()
        smtplib.starttls()
        smtplib.login(sender, 'nBbyzd2mj7DPwW7')
        smtplib.send_message(msg)

In [27]:
#Als nächstes schreibe ich eine Funktion für eine E-Mail im Falle, dass alle Links in Ordnung sind. 

def all_ok(sender, recipient):

    from email.message import EmailMessage
    import smtplib

#Header
    msg = EmailMessage()
    msg["Subject"] = "All links are ok - EOM"
    sender ='Theodor.sonnenschein@web.de' 
    msg['From'] = sender
    recipient = 'Theodor.sonnenschein@web.de'
    msg['To'] = recipient

#Nachricht
    msg.set_content(" ")


# Verbindung
    with smtplib.SMTP('smtp.web.de', 587) as s:
        s.ehlo()
        s.starttls()
        s.login(sender, 'nBbyzd2mj7DPwW7')
        s.send_message(msg)

In [38]:
#Anfang des eigentlichen Programms. Einlesen der csv-Datei mit Pandas, die vorher am bestimmten Ort abgespeichert wurde:

reading_list = pd.read_csv("~/2021-11-29-DL_Claudia_Wordsworth/Modul_6_Abschlussarbeit/all-links_checked.csv")

In [39]:
#isolieren der einzulesenden Spalte mit den URLs:
links = reading_list["Link"]
links

0    https://www.heise.de/invalid
1      https://www.gutenberg.org/
2          https://www.yahoo.com/
Name: Link, dtype: object

In [42]:
#Das Programm soll die URLs in der angegeben Spalte mit dem Modul "request" überprüfen und Ergebnisse in einer neuen Liste speichern:

broken_links = []

for url in links:
    try:
        r = requests.head(url)
        r.raise_for_status()
        
    # All Ausnahmen werden in einer Liste abgespeichert:
    except requests.exceptions.HTTPError as http_error:
        
           broken_links.append(f"HTTP error occurred: {http_error}")
            
    except requests.exceptions.RequestException as other_error:
            broken_links.append(f"Other error occurred: {other_error}")

#Nach Beenden des Überprüfens sollen die Ergebnisse per E-mail verschickt werden:
#Falls alle Links in Ordnung sind, wird eine einfache Info-E-mail verschickt:
if len(broken_links) == 0:
    all_ok('Theodor.sonnenschein@web.de', 'Theodor.sonnenschein@web.de')

#Gibt es Fehlermeldungen, wird die erstelle Liste in eine csv-Datei umgewandelt und 
#per E-Mail verschickt:

#df = pd.DataFrame([[links], [broken_links]], columns=["Link", "Error Code"])


pd.Series(list(broken_links)).to_csv("broken_links.csv")


send_email_alert('Theodor.sonnenschein@web.de', 'Theodor.sonnenschein@web.de', broken_links)